In [ ]:
try:
    import evidently
except:
    !npm install -g yarn
    !pip install git+https://github.com/evidentlyai/evidently.git@test_suite_alpha

In [ ]:
import pandas as pd
import numpy as np

from evidently.test_suite import TestSuite
from evidently.tests import TestNumberOfColumns
from evidently.tests import TestNumberOfDriftedFeatures
from evidently.tests import TestValueMAE, TestValueMeanError
from evidently.tests import TestAccuracyScore, TestPrecisionScore, TestF1Score, TestRecallScore, TestRocAuc, TestLogLoss, TestTPR, TestTNR, TestFPR, TestFNR
from evidently.tests import TestPrecisionByClass, TestRecallByClass, TestF1ByClass

### Creation for Test Suites with some tests (Simplified variant)

In [ ]:
from evidently import ColumnMapping
from datetime import datetime
from sklearn.datasets import fetch_openml

suite = TestSuite(tests=[
    TestNumberOfColumns(gt=1),
    # TestNumberOfDriftedFeatures(),
    TestValueMAE(),
    TestValueMeanError(),
])

data = fetch_openml(name='adult', version=2, as_frame='auto')
df = data.frame

ref = df[:20000]
curr = df[20000:]

curr['target'] = curr['education-num']
curr['preds'] = curr['education-num'].values + np.random.normal(0, 6, curr.shape[0])
ref['target'] = ref['education-num']
ref['preds'] = ref['education-num'].values + np.random.normal(0, 6, ref.shape[0])

# pd.DataFrame([{'a': 1, 'c': datetime.now(), 'b': 2}])
# pd.DataFrame([{'a': 2,'c': datetime.now(),  'b': None}, {'a': 1, 'b': 2}])
suite.run(reference_data=None,
          current_data=curr, column_mapping=ColumnMapping(target='target', prediction='preds'))
suite

In [ ]:
suite_with_reference = TestSuite(tests=[
    TestNumberOfColumns(gt=1),
    TestNumberOfDriftedFeatures(),
    TestValueMAE(),
    TestValueMeanError(),
])

data = fetch_openml(name='adult', version=2, as_frame='auto')
df = data.frame

ref = df[:20000]
curr = df[20000:]

curr['target'] = curr['education-num']
curr['preds'] = curr['education-num'].values + np.random.normal(0, 6, curr.shape[0])
ref['target'] = ref['education-num']
ref['preds'] = ref['education-num'].values + np.random.normal(0, 6, ref.shape[0])

# pd.DataFrame([{'a': 1, 'c': datetime.now(), 'b': 2}])
# pd.DataFrame([{'a': 2,'c': datetime.now(),  'b': None}, {'a': 1, 'b': 2}])
suite_with_reference.run(reference_data=ref,
          current_data=curr, column_mapping=ColumnMapping(target='target', prediction='preds'))
suite_with_reference

In [ ]:
from sklearn import datasets, ensemble, model_selection
bcancer = datasets.load_breast_cancer(as_frame=True)
bcancer_frame = bcancer.frame
bcancer_frame['target'] = bcancer.target
target = 'target'
prediction = 'prediction'

numerical_features = bcancer.feature_names
categorical_features = []

features = numerical_features.tolist() + categorical_features
train_data, test_data = model_selection.train_test_split(
    bcancer_frame, random_state=0)
model = ensemble.RandomForestClassifier(random_state=0)
model.fit(train_data[features], train_data.target)
train_predictions = model.predict(train_data[features])
test_predictions = model.predict(test_data[features])
train_data['prediction'] = [bcancer.target_names[x] for x in train_predictions]
test_data['prediction'] = [bcancer.target_names[x] for x in test_predictions]

train_data['target'] = [bcancer.target_names[x] for x in train_data['target']]
test_data['target'] = [bcancer.target_names[x] for x in test_data['target']]

train_data.reset_index(inplace=True, drop=True)
test_data.reset_index(inplace=True, drop=True)

train_data[bcancer.target_names] = model.predict_proba(train_data[features])
test_data[bcancer.target_names] = model.predict_proba(test_data[features])
bcancer_column_mapping = ColumnMapping()

bcancer_column_mapping.target = target
bcancer_column_mapping.prediction = prediction
bcancer_column_mapping.numerical_features = numerical_features


tests = [TestAccuracyScore(),
         TestPrecisionScore(),
         TestF1Score(),
         TestRecallScore(),
         TestTPR(),
         TestTNR(),
         TestFPR(),
         TestFNR(),
         *[TestPrecisionByClass(label=label) for label in bcancer.target_names],
         *[TestRecallByClass(label=label) for label in bcancer.target_names],
         *[TestF1ByClass(label=label) for label in bcancer.target_names],
]

suite_classification = TestSuite(tests=tests)

suite_classification.run(reference_data=None, current_data=test_data, column_mapping=bcancer_column_mapping)
suite_classification

In [ ]:
suite_classification_with_reference = TestSuite(tests=tests)

suite_classification_with_reference.run(reference_data=train_data, current_data=test_data, column_mapping=bcancer_column_mapping)
suite_classification_with_reference

In [ ]:
bcancer_column_mapping_probas = ColumnMapping()

bcancer_column_mapping_probas.target = target
bcancer_column_mapping_probas.prediction = bcancer.target_names.tolist()
bcancer_column_mapping_probas.numerical_features = numerical_features

suite_classification_probas = TestSuite(tests=tests + [TestRocAuc(), TestLogLoss()])

suite_classification_probas.run(reference_data=train_data, current_data=test_data, column_mapping=bcancer_column_mapping_probas)
suite_classification_probas

### Show default inline in notebook

### Show non-default in notebook

In [ ]:
suite.show(mode="inline")

In [ ]:
suite_with_reference.show(mode="inline")

### Save standalone html

In [ ]:
suite.save_html("example.html")

In [ ]:
suite_with_reference.save_html("example_with_reference.html")

In [ ]:
suite_classification.save_html("example_classification.html")
suite_classification_probas.save_html("example_classification_probas.html")

In [ ]:
suite_classification_with_reference.save_html("example_classification_with_reference.html")

In [ ]:
suite.json()

In [ ]:
suite_with_reference.json()

In [ ]:
suite_classification.json()

In [ ]:
suite_classification_with_reference.json()

In [ ]:
suite.save_json("example.json")